#### 사용자 정의 데이터셋
 - Pytorch에서 딥러닝 시 대량의 데이터 사용에 따른 부하(H/W, S/W) 및 많은 시간 소요에 대한 해결책으로 제시
 - 대량 데이터셋 전용 처리 모듈 제공
 - Dataset과 DataLoader
    * Dataset => 데이터셋 전터리, 텐서화 등의 작업 진행
    * DataLoader => Dataset 인스턴스를 사용해서 배치크기 만큼 데이터를 추출함
      +  Dataset이 있어야 dataLoader을 할수있다

[1] 모듈로딩 <hr>

In [1]:
# 모듈 로딩

import torch                        # 텐서 및 수치 계산 함수 관련 모듈
import torch.nn as nn               # 인공신경망 관련 모듈
import torch.nn.functional as F     # 손실, 거리 등 함수 관련 모듈
import torch.optim as optimizer     # 최적화 기법 관련 모듈
from torch.utils.data import DataLoader, Dataset
import pandas as pd                 # 데이터 파일 분석 관련 모듈
from sklearn.model_selection import train_test_split
from torchmetrics.regression import R2Score  # 성능 지표 관련 모듈 
from torchmetrics.classification import F1Score # 성능 지표 관련 모듈
from torchinfo import summary       # 모델 정보 관련 모듈
from sklearn.preprocessing import LabelEncoder



In [2]:
# 데이터
DATA_FILE = '../data/iris.csv'

In [3]:
# CSV => DF
irisDF = pd.read_csv(DATA_FILE)
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa


In [4]:
# 타겟 컬럼 수치화 ==>  LabelEncoder
encoder = LabelEncoder()
encoder.fit(irisDF['variety'])
irisDF['variety'] = encoder.transform(irisDF['variety'])

[2] 사용자 정의 데이터셋 클래스 생성
<hr>

In [6]:
#-------------------------------------------------------
# 클래스 목적 : 학습용 데이터셋 텐서화 및 전처리
# 클래스 이름 : CustomDataSet
# 부모 클래스 : torch.utils.data.dataset
# 매개 변수 : featureDF, targetDF
# >> 밖에서 전처리한걸 들고왔음
#-------------------------------------------------------


class CustomDataset(Dataset):
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드
    def __init__(self,featureDF, targetDF) :
        super().__init__()
        self.featureDF = featureDF 
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0]
        self.n_features = featureDF.shape[1]

    # 데이터의 개수 반환 메서드
    def __len__(self):
        return self.n_rows

    # 특정 index의 데이터와 타겟 반환 메서드 => Tensor 변환!!
    def __getitem__(self, idx):
        featureTS = torch.FloatTensor(self.featureDF.iloc[idx].values)
        targetTS = torch.FloatTensor(self.targetDF.iloc[idx].values)
        return featureTS, targetTS
        

In [ ]:
#-------------------------------------------------------
# 클래스 목적 : 파일 확장자별 데이터프레임 변환 기능
# 클래스 이름 : convertDataFrame
# 매개 변수 : data_path
# 함수 결과 : DataFrame
#-------------------------------------------------------

def convertDataFrame(data_path, exit_header=0):
    ext = data_path.rsplit('.')[-1]
    if ext == 'csv':
        return pd.read_csv(data_path, header = exit_header)
    elif ext == 'json':
        return pd.read_json(data_path, header = exit_header)
    elif ext in [ 'xlsx', 'xls']:
        return pd.read_excel(data_path, header = exit_header)
    else:
        return pd.read_table(data_path, header = exit_header)

In [ ]:
#-------------------------------------------------------
# 클래스기능 : 파일기반 데이터셋
# 클래스 명 : FileDataset
# 매개 변수 : data_path 파일 경로
# 부모 클래스 : utils.data.Dataset
# >> 파일을 받아서 위에서 확장자들 별로 달라지기에 1차적으로 데이터프레임을 빼오고
# 2. 데이터 프레임을 빼오고 이제 피쳐 라벨 분리하고,  지금 위아래 한세트
#-------------------------------------------------------  

class CustomDataset(Dataset):
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드
    def __init__(self,data_path) :
        super().__init__()
        
        # 데이터 파일 ==> DF로 변환 
        dataDF = convertDataFrame(data_path)
        
        # DF ==> 피쳐와 라벨 추출
        self.featuresDF = dataDF[dataDF.columns[:-1]]
        self.targetDF = dataDF[dataDF.columns[-1:]]

        self.n_features = self.featuresDF.shape[1]
        self.n_rows = self.featuresDF.shape[0]


    # 데이터의 개수 반환 메서드
    def __len__(self):
        return self.n_rows

    # 특정 index의 데이터와 타겟 반환 메서드 => Tensor 변환!!
    def __getitem__(self, idx):
        featureTS = torch.FloatTensor(self.featureDF.iloc[idx].values)
        targetTS = torch.FloatTensor(self.targetDF.iloc[idx].values)
        return featureTS, targetTS

[3] 데이터셋 인스턴스 생성 <hr>

In [7]:
# 피쳐와 타게 추출
featureDF, targetDF = irisDF[irisDF.columns[:-1]], irisDF[irisDF.columns[-1:]]  # irisDF[[irisDF.columns[-1]]]도 가능
print(f'featureDF => {featureDF.shape} , targetDF => {targetDF.shape}')

featureDF => (150, 4) , targetDF => (150, 1)


In [8]:
# iris 데이터셋 인스턴스 생성
irisDS = CustomDataset(featureDF, targetDF)

In [9]:
# iris 데이터셋 속성 >> [상자] 모양
irisDS.featureDF , irisDS.n_features, irisDS.n_rows

(     sepal.length  sepal.width  petal.length  petal.width
 0             5.1          3.5           1.4          0.2
 1             4.9          3.0           1.4          0.2
 2             4.7          3.2           1.3          0.2
 3             4.6          3.1           1.5          0.2
 4             5.0          3.6           1.4          0.2
 ..            ...          ...           ...          ...
 145           6.7          3.0           5.2          2.3
 146           6.3          2.5           5.0          1.9
 147           6.5          3.0           5.2          2.0
 148           6.2          3.4           5.4          2.3
 149           5.9          3.0           5.1          1.8
 
 [150 rows x 4 columns],
 4,
 150)

In [10]:
# iris 데이터셋 메서드
irisDS[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0.]))

[4] DataLoader 인스턴스 생성 <hr>

In [25]:
## 필요한 것 : Dataset 인스턴스, Batch_size
irisDL = DataLoader(irisDS, batch_size=3)

In [26]:
for datTS, targetTS in irisDL:
    print(datTS.shape, targetTS.shape)
    break


torch.Size([3, 4]) torch.Size([3, 1])
